# Data Loader Test

This notebook tests the data cleaning pipeline and generates a before/after comparison report.

In [ ]:
import sys
from pathlib import Path
from datetime import datetime
import pandas as pd

# Add project root to path
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

from src.data_loader import (
    load_raw_data,
    remove_corrupted_dates,
    remove_duplicates,
    filter_lyon_bbox,
    get_data_stats,
    LYON_BBOX
)

## 1. Load Raw Data

In [ ]:
df_raw = load_raw_data()
print(f"Raw data: {len(df_raw):,} rows")
df_raw.head()

## 2. Before Cleaning: Data Quality Issues

In [ ]:
# Check date component ranges
print("Date component ranges in raw data:")
print(f"  Year:   {df_raw['date_taken_year'].min()} - {df_raw['date_taken_year'].max()}")
print(f"  Month:  {df_raw['date_taken_month'].min()} - {df_raw['date_taken_month'].max()}")
print(f"  Day:    {df_raw['date_taken_day'].min()} - {df_raw['date_taken_day'].max()}")
print(f"  Hour:   {df_raw['date_taken_hour'].min()} - {df_raw['date_taken_hour'].max()}")

In [ ]:
# Count invalid date values
raw_issues = {
    'Invalid months (>12)': int((df_raw['date_taken_month'] > 12).sum()),
    'Invalid days (>31)': int((df_raw['date_taken_day'] > 31).sum()),
    'Invalid hours (>23)': int((df_raw['date_taken_hour'] > 23).sum()),
    'Invalid years (<1990 or >2025)': int(((df_raw['date_taken_year'] < 1990) | (df_raw['date_taken_year'] > 2025)).sum()),
    'Duplicate photos': len(df_raw) - df_raw['id'].nunique()
}

pd.DataFrame.from_dict(raw_issues, orient='index', columns=['Count'])

## 3. Apply Cleaning Pipeline

In [ ]:
# Step 1: Remove corrupted dates
df_step1 = remove_corrupted_dates(df_raw)
print(f"Step 1 - Remove corrupted dates: {len(df_raw):,} -> {len(df_step1):,} (removed {len(df_raw) - len(df_step1):,})")

# Step 2: Remove duplicates
df_step2, dup_removed = remove_duplicates(df_step1)
print(f"Step 2 - Remove duplicates:      {len(df_step1):,} -> {len(df_step2):,} (removed {dup_removed:,})")

# Step 3: Filter to Lyon bbox
df_clean = filter_lyon_bbox(df_step2)
print(f"Step 3 - Lyon bbox filter:       {len(df_step2):,} -> {len(df_clean):,} (removed {len(df_step2) - len(df_clean):,})")

print(f"\nFinal: {len(df_clean):,} rows ({len(df_clean)/len(df_raw)*100:.1f}% of original)")

## 4. After Cleaning: Data Quality Check

In [ ]:
# Check date component ranges after cleaning
print("Date component ranges after cleaning:")
print(f"  Year:   {df_clean['date_taken_year'].min()} - {df_clean['date_taken_year'].max()}")
print(f"  Month:  {df_clean['date_taken_month'].min()} - {df_clean['date_taken_month'].max()}")
print(f"  Day:    {df_clean['date_taken_day'].min()} - {df_clean['date_taken_day'].max()}")
print(f"  Hour:   {df_clean['date_taken_hour'].min()} - {df_clean['date_taken_hour'].max()}")

In [ ]:
# Final statistics
stats = get_data_stats(df_clean)
pd.DataFrame.from_dict(stats, orient='index', columns=['Value'])

## 5. Before vs After Comparison

In [ ]:
comparison = pd.DataFrame({
    'Before': [
        f"{len(df_raw):,}",
        f"{df_raw['id'].nunique():,}",
        f"{df_raw['user'].nunique():,}",
        f"{int(df_raw['date_taken_year'].min())} - {int(df_raw['date_taken_year'].max())}",
        f"{int(df_raw['date_taken_month'].min())} - {int(df_raw['date_taken_month'].max())}",
    ],
    'After': [
        f"{len(df_clean):,}",
        f"{stats['unique_photos']:,}",
        f"{stats['unique_users']:,}",
        f"{stats['year_range'][0]} - {stats['year_range'][1]}",
        f"{int(df_clean['date_taken_month'].min())} - {int(df_clean['date_taken_month'].max())}",
    ]
}, index=['Total rows', 'Unique photos', 'Unique users', 'Year range', 'Month range'])

comparison

## 6. Generate Report

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

report = f"""# Data Cleaning Report

**Generated:** {timestamp}

## Summary

| Metric | Before | After | Change |
|--------|--------|-------|--------|
| Total rows | {len(df_raw):,} | {len(df_clean):,} | -{len(df_raw) - len(df_clean):,} ({(len(df_raw) - len(df_clean)) / len(df_raw) * 100:.1f}%) |
| Unique photos | {df_raw['id'].nunique():,} | {stats['unique_photos']:,} | - |
| Unique users | {df_raw['user'].nunique():,} | {stats['unique_users']:,} | - |

## Cleaning Steps

1. **Remove corrupted dates:** {len(df_raw):,} → {len(df_step1):,} (removed {len(df_raw) - len(df_step1):,})
2. **Remove duplicates:** {len(df_step1):,} → {len(df_step2):,} (removed {dup_removed:,})
3. **Lyon bbox filter:** {len(df_step2):,} → {len(df_clean):,} (removed {len(df_step2) - len(df_clean):,})

## Date Range Comparison

| Component | Before | After |
|-----------|--------|-------|
| Year | {int(df_raw['date_taken_year'].min())} – {int(df_raw['date_taken_year'].max())} | {stats['year_range'][0]} – {stats['year_range'][1]} |
| Month | {int(df_raw['date_taken_month'].min())} – {int(df_raw['date_taken_month'].max())} | {int(df_clean['date_taken_month'].min())} – {int(df_clean['date_taken_month'].max())} |

## Conclusion

- ✅ All date values now valid
- ✅ No duplicate photos
- ✅ All coordinates in Lyon bbox
- ⚠️ {stats['empty_tags']:,} photos ({stats['empty_tags']/len(df_clean)*100:.0f}%) have no tags
- ⚠️ {stats['empty_titles']:,} photos ({stats['empty_titles']/len(df_clean)*100:.0f}%) have no title
"""

# Save report
report_path = PROJECT_ROOT / "reports" / "data_cleaning_report.md"
report_path.write_text(report)
print(f"Report saved to: {report_path}")

In [ ]:
from IPython.display import Markdown
Markdown(report)